In [5]:
import pickle
from albumentations import ShiftScaleRotate,Cutout,RandomContrast,RandomBrightness,Compose
from utility.albumentations_helper import create_transform
from utility.others import clear_all
import time
from functions import DataGenerator,PredictGenerator,create_model,dot_distance,exp_loss,\
                        sigmoid_dot_distance,dot_sigmoid_distance,cross_entropy_loss,l2_distance,\
                        margin_loss_fun_factory
from utility.grayscale_imagenet import Xception_greyscale
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import Xception,DenseNet121

In [2]:
color = True

### set up generator

In [3]:
if color:
    with open('/home/will/Desktop/kaggle/Whale/train_df_color.pkl', 'rb') as f:
        Ids_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_train_color.pkl', 'rb') as f:
        newWhale_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/val_df_color.pkl', 'rb') as f:
        Ids_val = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_val_color.pkl', 'rb') as f:
        newWhale_val = pickle.load(f)
else:
    with open('/home/will/Desktop/kaggle/Whale/train_df.pkl', 'rb') as f:
        Ids_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_train.pkl', 'rb') as f:
        newWhale_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/val_df.pkl', 'rb') as f:
        Ids_val = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_val.pkl', 'rb') as f:
        newWhale_val = pickle.load(f)

In [4]:
aug = Compose([RandomContrast(p=0.2),RandomBrightness(p=0.2),
                ShiftScaleRotate(shift_limit=0.03,rotate_limit=15,scale_limit=0.02,p=1),Cutout(p=0.5)])
transform = create_transform(aug)    

gen_train = DataGenerator(Ids_train,newWhale_train,transform,color=color,HalfBatch=8)
gen_val = DataGenerator(Ids_val,newWhale_val,transform,color=color,HalfBatch=8)

### build model

In [6]:
#conv_base = Xception(weights='imagenet',include_top=False,pooling='max')
conv_base = DenseNet121(weights='imagenet',include_top=False,pooling='max')
conv_base.trainable = False
# conv_base = Xception_greyscale((256,256,1),'max',False)

In [7]:
train_model,feature_model = create_model(1e-3,l2_distance,margin_loss_fun_factory(0.1,5),\
                                         conv_base,color)

### training

In [8]:
# conv_base = Xception(weights='imagenet',include_top=False,pooling='max')
# train_model,feature_model = create_model(1e-3,l2_distance,margin_loss_fun_factory(0.1,5),\
#                                         conv_base,color)
start = time.time()
history = train_model.fit_generator(
          gen_train,
          validation_data = gen_val,
          epochs=5,
          use_multiprocessing=True,workers=2,max_queue_size=20)
end = time.time()
print('time:{}'.format(end - start))

Epoch 1/5
714/714 [==============================] - 128s 179ms/step - loss: -0.4021 - val_loss: -0.4469
Epoch 2/5
714/714 [==============================] - 163s 228ms/step - loss: -0.4401 - val_loss: -0.4739
Epoch 3/5
714/714 [==============================] - 167s 234ms/step - loss: -0.4754 - val_loss: -0.4422
Epoch 4/5
714/714 [==============================] - 167s 234ms/step - loss: -0.4677 - val_loss: -0.4591
Epoch 5/5
714/714 [==============================] - 169s 236ms/step - loss: -0.4655 - val_loss: -0.4550
time:793.5516078472137


In [9]:
#conv_base = DenseNet121(weights='imagenet',include_top=False,pooling='max')
#train_model,feature_model = create_model(1e-3,l2_distance,margin_loss_fun_factory(0.1,5),\
#                                         conv_base,color)
start = time.time()
history = train_model.fit_generator(
          gen_train,
          validation_data = gen_val,
          epochs=5,
          use_multiprocessing=True,workers=2,max_queue_size=20)
end = time.time()
print('time:{}'.format(end - start))

Epoch 1/5
714/714 [==============================] - 157s 220ms/step - loss: -0.2326 - val_loss: -0.4480
Epoch 2/5
714/714 [==============================] - 147s 207ms/step - loss: -0.4798 - val_loss: -0.4687
Epoch 3/5
714/714 [==============================] - 135s 189ms/step - loss: -0.5311 - val_loss: -0.4971
Epoch 4/5
714/714 [==============================] - 143s 200ms/step - loss: -0.5399 - val_loss: -0.4593
Epoch 5/5
714/714 [==============================] - 141s 197ms/step - loss: -0.5498 - val_loss: -0.4553
time:723.2273600101471
